### Define which part you are developing

- EDA for electricity dataset
- Testing ARMIA model
- Developing LinearRegression model etc.

# Fossil Fuel & Renewable Energy Generation (TWh)

In [31]:
# Features to Extract:

# Natural Gas Power Generation (TWh)
# Coal Power Generation (TWh)
# Oil Power Generation (TWh)
# Hydro, Wind, Solar, Biofuels Power Generation (TWh)
# Total Electricity Generation (TWh)
# Electricity Imports (TWh) 


In [1]:
import pandas as pd 

In [2]:
ff_df = pd.read_csv('../data/raw/Generation by fuel-Thailand.csv')

In [3]:
ff_df

,generation,industry,timestamp,unit,region,2000,2001,2002,2003,2004,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Thailand,Hydro,Power Sector,2024-05-07 16:40:35.303458,TWh,5.89,6.17,7.37,7.21,5.90,...,5.41,5.16,3.76,3.54,4.69,7.60,6.31,4.54,4.54,6.60
1,Thailand,Wind,Power Sector,2024-05-07 16:40:35.303458,TWh,0.00,0.00,0.00,0.00,0.00,...,0.35,0.40,0.33,0.38,1.11,1.64,3.67,3.22,3.53,3.42
2,Thailand,Biofuels and waste,Power Sector,2024-05-07 16:40:35.303458,TWh,0.53,0.62,0.75,1.23,1.84,...,5.63,6.72,7.34,8.60,9.29,11.74,12.59,12.29,13.41,13.56
3,Thailand,Natural Gas,Power Sector,2024-05-07 16:40:35.303458,TWh,61.09,71.23,78.91,85.69,90.29,...,119.22,120.31,128.52,126.15,121.04,116.26,121.84,113.86,113.11,114.64
4,Thailand,Coal,Power Sector,2024-05-07 16:40:35.303458,TWh,18.20,20.30,19.29,19.30,20.40,...,35.35,37.57,34.58,37.11,35.73,35.80,35.83,36.82,36.06,35.52
5,Thailand,Solar,Power Sector,2024-05-07 16:40:35.303458,TWh,0.00,0.00,0.00,0.00,0.00,...,1.26,1.93,2.38,3.55,4.54,4.54,5.15,5.03,4.99,4.88
6,Thailand,Electricity imports,Power Sector,2024-05-07 16:40:35.303458,TWh,2.82,2.71,2.62,2.27,3.12,...,11.29,10.19,12.15,18.39,23.32,25.58,22.66,26.93,31.39,32.20


#  CO₂ Emissions from Power Generation (MtCO₂)

Total CO₂ Emissions from Power Generation (MtCO₂) ✅ (Target Variable)

In [179]:
co2_emissions_powergeneration_df = pd.read_csv('../data/raw/Power emissions by fuel-Thailand.csv')

In [180]:
co2_emissions_powergeneration_df.drop(columns='unit')

,generation,industry,timestamp,region,2000,2001,2002,2003,2004,2005,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Thailand,Natural Gas,Power Sector,MtCO2,34.95,40.75,45.14,49.02,51.65,54.04,...,68.82,73.52,72.16,69.24,66.51,69.70,65.13,64.70,65.58,74.02
1,Thailand,Hydro,Power Sector,MtCO2,0.14,0.15,0.18,0.17,0.14,0.13,...,0.12,0.09,0.08,0.11,0.18,0.15,0.11,0.11,0.16,0.16
2,Thailand,Coal,Power Sector,MtCO2,16.47,18.36,17.45,17.46,18.45,18.64,...,33.06,30.43,32.66,31.44,31.50,31.42,32.19,31.52,31.05,26.61
3,Thailand,Biofuels and waste,Power Sector,MtCO2,0.11,0.13,0.16,0.27,0.40,0.40,...,1.45,1.59,1.86,2.01,2.54,2.72,2.66,2.90,2.93,3.02
4,Thailand,Wind,Power Sector,MtCO2,0.00,0.00,0.00,0.00,0.00,0.00,...,0.01,0.00,0.00,0.01,0.02,0.05,0.04,0.04,0.04,0.05
5,Thailand,Solar,Power Sector,MtCO2,0.00,0.00,0.00,0.00,0.00,0.00,...,0.09,0.11,0.17,0.22,0.22,0.24,0.24,0.24,0.23,0.24


In [181]:
co2_emissions_powergeneration_df = co2_emissions_powergeneration_df.transpose()

In [182]:
co2_emissions_powergeneration_df

,0,1,2,3,4,5
generation,Thailand,Thailand,Thailand,Thailand,Thailand,Thailand
industry,Natural Gas,Hydro,Coal,Biofuels and waste,Wind,Solar
timestamp,Power Sector,Power Sector,Power Sector,Power Sector,Power Sector,Power Sector
unit,2024-05-08 17:47:59.510116,2024-05-08 17:47:59.510116,2024-05-08 17:47:59.510116,2024-05-08 17:47:59.510116,2024-05-08 17:47:59.510116,2024-05-08 17:47:59.510116
region,MtCO2,MtCO2,MtCO2,MtCO2,MtCO2,MtCO2
2000,34.95,0.14,16.47,0.11,0.0,0.0
2001,40.75,0.15,18.36,0.13,0.0,0.0
2002,45.14,0.18,17.45,0.16,0.0,0.0
2003,49.02,0.17,17.46,0.27,0.0,0.0
2004,51.65,0.14,18.45,0.4,0.0,0.0


In [183]:
co2_emissions_powergeneration_df = co2_emissions_powergeneration_df.reset_index()

In [184]:
co2_emissions_powergeneration_df

,index,0,1,2,3,4,5
0,generation,Thailand,Thailand,Thailand,Thailand,Thailand,Thailand
1,industry,Natural Gas,Hydro,Coal,Biofuels and waste,Wind,Solar
2,timestamp,Power Sector,Power Sector,Power Sector,Power Sector,Power Sector,Power Sector
3,unit,2024-05-08 17:47:59.510116,2024-05-08 17:47:59.510116,2024-05-08 17:47:59.510116,2024-05-08 17:47:59.510116,2024-05-08 17:47:59.510116,2024-05-08 17:47:59.510116
4,region,MtCO2,MtCO2,MtCO2,MtCO2,MtCO2,MtCO2
5,2000,34.95,0.14,16.47,0.11,0.0,0.0
6,2001,40.75,0.15,18.36,0.13,0.0,0.0
7,2002,45.14,0.18,17.45,0.16,0.0,0.0
8,2003,49.02,0.17,17.46,0.27,0.0,0.0
9,2004,51.65,0.14,18.45,0.4,0.0,0.0


In [185]:
co2_emissions_powergeneration_df.rename(columns={"index": "Year"}, inplace=True)

In [186]:
co2_emissions_powergeneration_df

,Year,0,1,2,3,4,5
0,generation,Thailand,Thailand,Thailand,Thailand,Thailand,Thailand
1,industry,Natural Gas,Hydro,Coal,Biofuels and waste,Wind,Solar
2,timestamp,Power Sector,Power Sector,Power Sector,Power Sector,Power Sector,Power Sector
3,unit,2024-05-08 17:47:59.510116,2024-05-08 17:47:59.510116,2024-05-08 17:47:59.510116,2024-05-08 17:47:59.510116,2024-05-08 17:47:59.510116,2024-05-08 17:47:59.510116
4,region,MtCO2,MtCO2,MtCO2,MtCO2,MtCO2,MtCO2
5,2000,34.95,0.14,16.47,0.11,0.0,0.0
6,2001,40.75,0.15,18.36,0.13,0.0,0.0
7,2002,45.14,0.18,17.45,0.16,0.0,0.0
8,2003,49.02,0.17,17.46,0.27,0.0,0.0
9,2004,51.65,0.14,18.45,0.4,0.0,0.0


In [187]:
co2_emissions_powergeneration_df = co2_emissions_powergeneration_df.drop([0,2,3,4])

In [188]:
co2_emissions_powergeneration_df

,Year,0,1,2,3,4,5
1,industry,Natural Gas,Hydro,Coal,Biofuels and waste,Wind,Solar
5,2000,34.95,0.14,16.47,0.11,0.0,0.0
6,2001,40.75,0.15,18.36,0.13,0.0,0.0
7,2002,45.14,0.18,17.45,0.16,0.0,0.0
8,2003,49.02,0.17,17.46,0.27,0.0,0.0
9,2004,51.65,0.14,18.45,0.4,0.0,0.0
10,2005,54.04,0.13,18.64,0.4,0.0,0.0
11,2006,54.02,0.19,22.04,0.43,0.0,0.0
12,2007,56.2,0.19,27.62,0.52,0.0,0.0
13,2008,59.76,0.17,27.51,0.46,0.0,0.0


In [190]:
co2_emissions_powergeneration_df.dtypes

Year    object
0       object
1       object
2       object
3       object
4       object
5       object
dtype: object

In [191]:
co2_emissions_powergeneration_df.columns = co2_emissions_powergeneration_df.iloc[0]  
co2_emissions_powergeneration_df = co2_emissions_powergeneration_df.iloc[1:].reset_index(drop=True)

In [193]:
co2_emissions_powergeneration_df.rename(columns={co2_emissions_powergeneration_df.columns[0]: "Year"}, inplace=True)

In [194]:
co2_emissions_powergeneration_df["Year"] = co2_emissions_powergeneration_df["Year"].astype(int)

In [197]:
co2_emissions_powergeneration_df

1,Year,Natural Gas,Hydro,Coal,Biofuels and waste,Wind,Solar
0,2000,34.95,0.14,16.47,0.11,0.0,0.0
1,2001,40.75,0.15,18.36,0.13,0.0,0.0
2,2002,45.14,0.18,17.45,0.16,0.0,0.0
3,2003,49.02,0.17,17.46,0.27,0.0,0.0
4,2004,51.65,0.14,18.45,0.4,0.0,0.0
5,2005,54.04,0.13,18.64,0.4,0.0,0.0
6,2006,54.02,0.19,22.04,0.43,0.0,0.0
7,2007,56.2,0.19,27.62,0.52,0.0,0.0
8,2008,59.76,0.17,27.51,0.46,0.0,0.0
9,2009,60.83,0.17,26.59,0.49,0.0,0.0


In [200]:
co2_emissions_powergeneration_df = co2_emissions_powergeneration_df[co2_emissions_powergeneration_df.Year >= 2002]

In [202]:
co2_emissions_powergeneration_df.columns

Index(['Year', 'Natural Gas', 'Hydro', 'Coal', 'Biofuels and waste', 'Wind',
       'Solar'],
      dtype='object', name=1)

In [204]:
columns = ['Natural Gas', 'Hydro', "Coal", 'Biofuels and waste', 'Wind', 'Solar']

for col in columns:
    co2_emissions_powergeneration_df.rename(columns={col: f"{col} (MtCO2)"}, inplace=True)

/tmp/ipykernel_1140294/2099809807.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  co2_emissions_powergeneration_df.rename(columns={col: f"{col} (MtCO2)"}, inplace=True)


In [205]:
co2_emissions_powergeneration_df

1,Year,Natural Gas (MtCO2),Hydro (MtCO2),Coal (MtCO2),Biofuels and waste (MtCO2),Wind (MtCO2),Solar (MtCO2)
2,2002,45.14,0.18,17.45,0.16,0.0,0.0
3,2003,49.02,0.17,17.46,0.27,0.0,0.0
4,2004,51.65,0.14,18.45,0.4,0.0,0.0
5,2005,54.04,0.13,18.64,0.4,0.0,0.0
6,2006,54.02,0.19,22.04,0.43,0.0,0.0
7,2007,56.2,0.19,27.62,0.52,0.0,0.0
8,2008,59.76,0.17,27.51,0.46,0.0,0.0
9,2009,60.83,0.17,26.59,0.49,0.0,0.0
10,2010,67.75,0.13,26.54,0.73,0.0,0.0
11,2011,61.93,0.19,28.27,0.86,0.0,0.0


# Grid Emission Factor (tCO₂/MWh)

In [6]:
ge_df = pd.read_csv('../data/raw/Emission Intensity of Power Generation-Thailand.csv')

In [7]:
ge_df.head()

,generation,industry,timestamp,unit,region,2000,2001,2002,2003,2004,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Thailand,Grid emission factor,Power Sector,2024-05-10 12:25:03.480602,tCO2_MWh,0.608,0.606,0.593,0.591,0.599,...,0.602,0.598,0.596,0.584,0.569,0.563,0.571,0.567,0.561,0.55


# Global & Regional Energy Trends

In [8]:
energy_trends_df = pd.read_csv("../data/raw/emberChartData.csv", sep=";", skiprows=1)


In [9]:
energy_trends_df

,country_or_region,year,variable,generation_twh
0,Thailand,2000,Bioenergy,0.53
1,Thailand,2000,Clean,6.42
2,Thailand,2000,Coal,18.20
3,Thailand,2000,Fossil,89.10
4,Thailand,2000,Gas,61.09
...,...,...,...,...
235,Thailand,2023,Hydro,6.59
236,Thailand,2023,Other Fossil,0.89
237,Thailand,2023,Solar,5.42
238,Thailand,2023,Wind,3.61


# Deforestation & Land Use (Related to Bioenergy)

Annual Tree Cover Loss (ha) → Impacts biofuels and carbon absorption.

In [91]:
deforestation_df = pd.read_csv('../data/raw/treecover_loss__ha.csv')

In [92]:
deforestation_df.drop(columns=['iso'], inplace=True)

In [93]:
deforestation_df.rename(columns={'umd_tree_cover_loss__year': 'Year',
                                 'umd_tree_cover_loss__ha':'UMD_Tree cover loss (ha)',
                                 'gfw_gross_emissions_co2e_all_gases__Mg': 'GFW_Gross emissions co2 (Mg)'}, inplace=True)

In [94]:
deforestation_df

,Year,UMD_Tree cover loss (ha),GFW_Gross emissions co2 (Mg)
0,2001,36101.943587,1.836831e+07
1,2002,60882.189737,3.119673e+07
2,2003,58447.942201,2.984648e+07
3,2004,91835.294338,4.718033e+07
4,2005,84287.060043,4.508730e+07
5,2006,99820.579633,5.214129e+07
6,2007,92628.558641,4.877112e+07
7,2008,80143.317979,4.341977e+07
8,2009,132690.267032,7.236850e+07
9,2010,134984.309304,7.490822e+07


# Thailand CO2 emission (world data bank)

In [12]:
co2_df = pd.read_csv('../data/raw/co-emissions-per-capita.csv')

In [13]:
co2_df.head()

,Entity,Code,Year,Annual CO₂ emissions (per capita)
0,Afghanistan,AFG,1949,0.001992
1,Afghanistan,AFG,1950,0.010837
2,Afghanistan,AFG,1951,0.011625
3,Afghanistan,AFG,1952,0.011468
4,Afghanistan,AFG,1953,0.013123


# GDP rate

In [15]:
# gdp_df.head()

In [235]:
gdp_df = pd.read_csv('../data/raw/MKTGDPTHA646NWDB (1).csv')

In [236]:
gdp_df.head()

,observation_date,MKTGDPTHA646NWDB
0,2002-01-01,1.343009e+11
1,2003-01-01,1.522806e+11
2,2004-01-01,1.728957e+11
3,2005-01-01,1.893184e+11
4,2006-01-01,2.217583e+11


In [237]:
gdp_df = gdp_df.rename(columns={'MKTGDPTHA646NWDB': 'GDP (USD)',
                       'observation_date': 'Year'})

In [238]:
gdp_df.columns.to_list()

['Year', 'GDP (USD)']

In [239]:
gdp_df["Year"] = pd.to_datetime(gdp_df["Year"])

gdp_df["Year"] = gdp_df["Year"].dt.year

In [240]:
gdp_df["Year"] = gdp_df["Year"].astype(int)

In [241]:
gdp_df.dtypes

Year           int64
GDP (USD)    float64
dtype: object

In [242]:
gdp_df

,Year,GDP (USD)
0,2002,1.343009e+11
1,2003,1.522806e+11
2,2004,1.728957e+11
3,2005,1.893184e+11
4,2006,2.217583e+11
5,2007,2.629426e+11
6,2008,2.913830e+11
7,2009,2.817106e+11
8,2010,3.411048e+11
9,2011,3.708187e+11


# power generation by type

In [18]:
power_generation_df = pd.read_csv("../data/raw/POWER GENERATION BY TYPE.csv", skiprows=5, skipfooter=6)  
power_generation_df.head()

/tmp/ipykernel_1140294/1349196458.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  power_generation_df = pd.read_csv("../data/raw/POWER GENERATION BY TYPE.csv", skiprows=5, skipfooter=6)


,YEAR,HYDRO,FUEL OIL,LIGNITE,GAS,DIESEL,THERMAL,NRE*,IMPORTED,DEDP,SPP,IPP,VSPP,TOTAL
0,1986,"5,531.70","3,334.88","5,541.28","10,251.71",10.24,NaN,NaN,758.38,NaN,NaN,NaN,NaN,"25,428.19"
1,1987,"4,056.23","2,188.38","6,698.01","15,623.66",2.41,NaN,NaN,415.45,4.92,NaN,NaN,NaN,"28,989.06"
2,1988,"3,718.17","3,142.03","6,799.50","18,719.88",3.47,NaN,NaN,429.78,30.19,NaN,NaN,NaN,"32,843.02"
3,1989,"5,512.21","4,738.83","7,869.57","19,194.75",16.45,NaN,NaN,643.15,21.57,NaN,NaN,NaN,"37,996.53"
4,1990,"4,900.11","10,012.63","11,052.85","17,765.06",356.85,1.01,0.02,652.32,23.98,NaN,NaN,NaN,"44,764.83"


In [19]:
power_generation_df.head()

,YEAR,HYDRO,FUEL OIL,LIGNITE,GAS,DIESEL,THERMAL,NRE*,IMPORTED,DEDP,SPP,IPP,VSPP,TOTAL
0,1986,"5,531.70","3,334.88","5,541.28","10,251.71",10.24,NaN,NaN,758.38,NaN,NaN,NaN,NaN,"25,428.19"
1,1987,"4,056.23","2,188.38","6,698.01","15,623.66",2.41,NaN,NaN,415.45,4.92,NaN,NaN,NaN,"28,989.06"
2,1988,"3,718.17","3,142.03","6,799.50","18,719.88",3.47,NaN,NaN,429.78,30.19,NaN,NaN,NaN,"32,843.02"
3,1989,"5,512.21","4,738.83","7,869.57","19,194.75",16.45,NaN,NaN,643.15,21.57,NaN,NaN,NaN,"37,996.53"
4,1990,"4,900.11","10,012.63","11,052.85","17,765.06",356.85,1.01,0.02,652.32,23.98,NaN,NaN,NaN,"44,764.83"


In [20]:
power_generation_df = power_generation_df[['YEAR', 'TOTAL']]

In [21]:
power_generation_df.rename(columns={'YEAR':'Year', 'TOTAL':'Total power generation'}, inplace=True)

In [22]:
power_generation_df.dtypes

Year                       int64
Total power generation    object
dtype: object

In [23]:
power_generation_df = power_generation_df[power_generation_df['Year'] >= 2002]

In [24]:
power_generation_df

,Year,Total power generation
16,2002,"111,253.87"
17,2003,"118,411.44"
18,2004,"127,510.51"
19,2005,"134,798.20"
20,2006,"141,918.57"
21,2007,"147,025.89"
22,2008,"148,220.93"
23,2009,"148,358.11"
24,2010,"164,828.81"
25,2011,"164,089.92"


# Electricity consumption by whole country

In [25]:
electricity_consumption_df = pd.read_csv('../data/raw/T05_03_04-2 (2).csv', skiprows=3, skipfooter=4)

/tmp/ipykernel_1140294/1927585605.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  electricity_consumption_df = pd.read_csv('../data/raw/T05_03_04-2 (2).csv', skiprows=3, skipfooter=4)


In [26]:
electricity_consumption_df.head()

,Year,Residential,Business,Industrial,Government and Non-Profit,Agriculture,Other,Total,Free of Charge,Grand Total
0,2002,"21,963","19,268","52,965","3,635",192,"1,109","99,132",959,"100,091"
1,2003,"23,330","20,684","56,747","3,701",228,"1,214","105,903","1,084","106,987"
2,2004,"24,538","22,090","61,652","3,814",251,"1,572","113,917","1,184","115,101"
3,2005,"25,482","23,145","65,365","3,848",249,"1,885","119,976","1,264","121,240"
4,2006,"26,847","24,818","68,362","3,979",240,"2,097","126,343","1,536","127,879"


In [27]:
electricity_consumption_df.rename(columns={'Grand Total':'Total electricity consumption'}, inplace=True)

In [28]:
electricity_consumption_df = electricity_consumption_df[['Year', 'Total electricity consumption']]

In [29]:
electricity_consumption_df

,Year,Total electricity consumption
0,2002,"100,091"
1,2003,"106,987"
2,2004,"115,101"
3,2005,"121,240"
4,2006,"127,879"
5,2007,"133,113"
6,2008,"135,520"
7,2009,"135,181"
8,2010,"149,301"
9,2011,"148,855"


In [37]:
power_electricity_df = pd.merge(power_generation_df, electricity_consumption_df, on='Year')

In [38]:
power_electricity_df = power_electricity_df[power_electricity_df['Year'] <= 2023]

In [39]:
power_electricity_df

,Year,Total power generation,Total electricity consumption
0,2002,"111,253.87","100,091"
1,2003,"118,411.44","106,987"
2,2004,"127,510.51","115,101"
3,2005,"134,798.20","121,240"
4,2006,"141,918.57","127,879"
5,2007,"147,025.89","133,113"
6,2008,"148,220.93","135,520"
7,2009,"148,358.11","135,181"
8,2010,"164,828.81","149,301"
9,2011,"164,089.92","148,855"


# PM 2.5 

In [ ]:
# import requests
# import json

# API_KEY = "de06c48c-da34-406d-b8df-50e8564aa339"  
# def get_nearest_city_data(lat, lng):
#     url = f"http://api.airvisual.com/v2/nearest_city?lat={lat}&lon={lng}&key={API_KEY}"
#     response = requests.get(url)
#     return response.json()

# def get_city_data(city, state, country="Thailand"):
#     url = f"http://api.airvisual.com/v2/city?city={city}&state={state}&country={country}&key={API_KEY}"
#     response = requests.get(url)
#     return response.json()

# def get_states_in_thailand():
#     url = f"http://api.airvisual.com/v2/states?country=Thailand&key={API_KEY}"
#     response = requests.get(url)
#     return response.json()

# def get_cities_in_state(state, country="Thailand"):
#     url = f"http://api.airvisual.com/v2/cities?state={state}&country={country}&key={API_KEY}"
#     response = requests.get(url)
#     return response.json()

# bangkok_data = get_city_data("Bangkok", "Bangkok")
# print(json.dumps(bangkok_data, indent=2))

# if bangkok_data.get("status") == "success":
#     pollution_data = bangkok_data["data"]["current"]["pollution"]
#     pm25 = pollution_data.get("aqius")  # US AQI based on PM2.5
#     pm25_concentration = pollution_data.get("pm25")  # Actual PM2.5 concentration in μg/m³
    
#     print(f"Bangkok PM2.5 US AQI: {pm25}")
#     if pm25_concentration is not None:
#         print(f"Bangkok PM2.5 Concentration: {pm25_concentration} μg/m³")
#     else:
#         print("PM2.5 concentration data not available")

{
  "status": "success",
  "data": {
    "city": "Bangkok",
    "state": "Bangkok",
    "country": "Thailand",
    "location": {
      "type": "Point",
      "coordinates": [
        100.4888394,
        13.7292915
      ]
    },
    "current": {
      "pollution": {
        "ts": "2025-03-18T07:00:00.000Z",
        "aqius": 94,
        "mainus": "p2",
        "aqicn": 53,
        "maincn": "p1"
      },
      "weather": {
        "ts": "2025-03-18T07:00:00.000Z",
        "tp": 37,
        "pr": 1010,
        "hu": 47,
        "ws": 4.07,
        "wd": 184,
        "ic": "04d"
      }
    }
  }
}
Bangkok PM2.5 US AQI: 94
PM2.5 concentration data not available


# production index

The industrial production index (IPI) measures levels of production in the manufacturing, mining—including oil and gas field drilling services—and electrical and gas utilities sectors. It also measures capacity, an estimate of the production levels that could be sustainably maintained; and capacity utilization, the ratio of actual output to capacity.

# CO2 emissions (carbon atlas)

In [71]:
carbon_atlas_df = pd.read_csv('../data/raw/export_emissions.csv', skiprows=1, skipfooter=3)

/tmp/ipykernel_1140294/2721725317.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  carbon_atlas_df = pd.read_csv('../data/raw/export_emissions.csv', skiprows=1, skipfooter=3)


In [72]:
carbon_atlas_df.head()

,Unnamed: 0,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Wallis and Futuna Islands,Western Sahara,Yemen,Zambia,Zimbabwe
0,2000,1.0471,3.0249,85.3986,0.52395,15.9951,0.076944,0.35541,143.4687,3.4914,...,5.2680,123.4773,0.084272,142.8349,53.9355,0.025648,NaN,15.7253,1.7841,13.8145
1,2001,1.0691,3.2207,86.8393,0.52395,15.9085,0.084272,0.37006,135.1368,3.5327,...,5.0304,126.2212,0.080608,130.5339,60.9838,0.025648,NaN,17.0773,1.8856,12.5087
2,2002,1.3410,3.7483,90.6924,0.53128,16.0800,0.084272,0.40670,124.9310,3.0768,...,4.5578,132.5752,0.080608,164.1529,69.5278,0.025648,NaN,16.5814,1.9609,11.8958
3,2003,1.5596,4.3035,94.5806,0.53494,17.4844,0.087936,0.42869,134.4812,3.4556,...,4.5269,130.2944,0.080608,154.2622,77.2616,0.025648,NaN,18.4816,2.0744,10.6096
4,2004,1.2372,4.1767,91.7142,0.56059,17.0157,0.102590,0.45067,157.0494,3.6912,...,5.5648,129.0976,0.073280,142.7669,88.9392,0.025648,NaN,19.9772,2.1041,9.4286


In [73]:
carbon_atlas_df.columns

Index(['Unnamed: 0', 'Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia',
       ...
       'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela', 'Vietnam',
       'Wallis and Futuna Islands', 'Western Sahara', 'Yemen', 'Zambia',
       'Zimbabwe'],
      dtype='object', length=222)

In [74]:
carbon_atlas_df.rename(columns={'Unnamed: 0': 'Year'}, inplace=True)

In [75]:
'Thailand' in carbon_atlas_df.columns

True

In [76]:
carbon_atlas_df.columns.get_loc('Thailand')

197

In [77]:
carbon_atlas_df = carbon_atlas_df[['Thailand', 'Year']]

In [78]:
carbon_atlas_df.dtypes

Thailand    float64
Year          int64
dtype: object

In [79]:
carbon_atlas_df.rename(columns={'Thailand': 'Carbon Emissions(MtCO2)'}, inplace=True)

In [80]:
carbon_atlas_df = carbon_atlas_df[carbon_atlas_df['Year'] >= 2002]

In [81]:
year_col = carbon_atlas_df.pop('Year')
carbon_atlas_df.insert(0, 'Year', year_col)

In [82]:
carbon_atlas_df

,Year,Carbon Emissions(MtCO2)
2,2002,184.6655
3,2003,191.0877
4,2004,207.3307
5,2005,214.3251
6,2006,215.8247
7,2007,224.0826
8,2008,224.2487
9,2009,228.8362
10,2010,241.0190
11,2011,244.8782


# Combined data

In [83]:
power_electricity_df

,Year,Total power generation,Total electricity consumption
0,2002,"111,253.87","100,091"
1,2003,"118,411.44","106,987"
2,2004,"127,510.51","115,101"
3,2005,"134,798.20","121,240"
4,2006,"141,918.57","127,879"
5,2007,"147,025.89","133,113"
6,2008,"148,220.93","135,520"
7,2009,"148,358.11","135,181"
8,2010,"164,828.81","149,301"
9,2011,"164,089.92","148,855"


In [84]:
power_electricity_carbon = pd.merge(power_electricity_df, carbon_atlas_df,  on='Year')

In [85]:
power_electricity_carbon.rename(columns={'Total electricity consumption': 'Total electricity consumption (GWh)', 
                                         'Total power generation': 'Total power generation (GWh)'}, inplace = True)

In [86]:
power_electricity_carbon

,Year,Total power generation (GWh),Total electricity consumption (GWh),Carbon Emissions(MtCO2)
0,2002,"111,253.87","100,091",184.6655
1,2003,"118,411.44","106,987",191.0877
2,2004,"127,510.51","115,101",207.3307
3,2005,"134,798.20","121,240",214.3251
4,2006,"141,918.57","127,879",215.8247
5,2007,"147,025.89","133,113",224.0826
6,2008,"148,220.93","135,520",224.2487
7,2009,"148,358.11","135,181",228.8362
8,2010,"164,828.81","149,301",241.0190
9,2011,"164,089.92","148,855",244.8782


In [207]:
power_electricity_carbon_deforestation = pd.merge(power_electricity_carbon, deforestation_df,  on='Year')

In [208]:
power_electricity_carbon_deforestation

,Year,Total power generation (GWh),Total electricity consumption (GWh),Carbon Emissions(MtCO2),UMD_Tree cover loss (ha),GFW_Gross emissions co2 (Mg)
0,2002,"111,253.87","100,091",184.6655,60882.189737,3.119673e+07
1,2003,"118,411.44","106,987",191.0877,58447.942201,2.984648e+07
2,2004,"127,510.51","115,101",207.3307,91835.294338,4.718033e+07
3,2005,"134,798.20","121,240",214.3251,84287.060043,4.508730e+07
4,2006,"141,918.57","127,879",215.8247,99820.579633,5.214129e+07
5,2007,"147,025.89","133,113",224.0826,92628.558641,4.877112e+07
6,2008,"148,220.93","135,520",224.2487,80143.317979,4.341977e+07
7,2009,"148,358.11","135,181",228.8362,132690.267032,7.236850e+07
8,2010,"164,828.81","149,301",241.0190,134984.309304,7.490822e+07
9,2011,"164,089.92","148,855",244.8782,94544.693780,5.146751e+07


In [209]:
co2_emissions_powergeneration_df

1,Year,Natural Gas (MtCO2),Hydro (MtCO2),Coal (MtCO2),Biofuels and waste (MtCO2),Wind (MtCO2),Solar (MtCO2)
2,2002,45.14,0.18,17.45,0.16,0.0,0.0
3,2003,49.02,0.17,17.46,0.27,0.0,0.0
4,2004,51.65,0.14,18.45,0.4,0.0,0.0
5,2005,54.04,0.13,18.64,0.4,0.0,0.0
6,2006,54.02,0.19,22.04,0.43,0.0,0.0
7,2007,56.2,0.19,27.62,0.52,0.0,0.0
8,2008,59.76,0.17,27.51,0.46,0.0,0.0
9,2009,60.83,0.17,26.59,0.49,0.0,0.0
10,2010,67.75,0.13,26.54,0.73,0.0,0.0
11,2011,61.93,0.19,28.27,0.86,0.0,0.0


In [210]:
power_electricity_carbon_deforestation_co2_emisssions = pd.merge(power_electricity_carbon_deforestation, co2_emissions_powergeneration_df, on='Year')

In [211]:
power_electricity_carbon_deforestation_co2_emisssions

,Year,Total power generation (GWh),Total electricity consumption (GWh),Carbon Emissions(MtCO2),UMD_Tree cover loss (ha),GFW_Gross emissions co2 (Mg),Natural Gas (MtCO2),Hydro (MtCO2),Coal (MtCO2),Biofuels and waste (MtCO2),Wind (MtCO2),Solar (MtCO2)
0,2002,"111,253.87","100,091",184.6655,60882.189737,3.119673e+07,45.14,0.18,17.45,0.16,0.0,0.0
1,2003,"118,411.44","106,987",191.0877,58447.942201,2.984648e+07,49.02,0.17,17.46,0.27,0.0,0.0
2,2004,"127,510.51","115,101",207.3307,91835.294338,4.718033e+07,51.65,0.14,18.45,0.4,0.0,0.0
3,2005,"134,798.20","121,240",214.3251,84287.060043,4.508730e+07,54.04,0.13,18.64,0.4,0.0,0.0
4,2006,"141,918.57","127,879",215.8247,99820.579633,5.214129e+07,54.02,0.19,22.04,0.43,0.0,0.0
5,2007,"147,025.89","133,113",224.0826,92628.558641,4.877112e+07,56.2,0.19,27.62,0.52,0.0,0.0
6,2008,"148,220.93","135,520",224.2487,80143.317979,4.341977e+07,59.76,0.17,27.51,0.46,0.0,0.0
7,2009,"148,358.11","135,181",228.8362,132690.267032,7.236850e+07,60.83,0.17,26.59,0.49,0.0,0.0
8,2010,"164,828.81","149,301",241.0190,134984.309304,7.490822e+07,67.75,0.13,26.54,0.73,0.0,0.0
9,2011,"164,089.92","148,855",244.8782,94544.693780,5.146751e+07,61.93,0.19,28.27,0.86,0.0,0.0


In [243]:
power_electricity_carbon_deforestation_co2_emisssions_gdp = pd.merge(power_electricity_carbon_deforestation_co2_emisssions, gdp_df, on='Year')

In [244]:
power_electricity_carbon_deforestation_co2_emisssions_gdp

,Year,Total power generation (GWh),Total electricity consumption (GWh),Carbon Emissions(MtCO2),UMD_Tree cover loss (ha),GFW_Gross emissions co2 (Mg),Natural Gas (MtCO2),Hydro (MtCO2),Coal (MtCO2),Biofuels and waste (MtCO2),Wind (MtCO2),Solar (MtCO2),GDP (USD)
0,2002,"111,253.87","100,091",184.6655,60882.189737,3.119673e+07,45.14,0.18,17.45,0.16,0.0,0.0,1.343009e+11
1,2003,"118,411.44","106,987",191.0877,58447.942201,2.984648e+07,49.02,0.17,17.46,0.27,0.0,0.0,1.522806e+11
2,2004,"127,510.51","115,101",207.3307,91835.294338,4.718033e+07,51.65,0.14,18.45,0.4,0.0,0.0,1.728957e+11
3,2005,"134,798.20","121,240",214.3251,84287.060043,4.508730e+07,54.04,0.13,18.64,0.4,0.0,0.0,1.893184e+11
4,2006,"141,918.57","127,879",215.8247,99820.579633,5.214129e+07,54.02,0.19,22.04,0.43,0.0,0.0,2.217583e+11
5,2007,"147,025.89","133,113",224.0826,92628.558641,4.877112e+07,56.2,0.19,27.62,0.52,0.0,0.0,2.629426e+11
6,2008,"148,220.93","135,520",224.2487,80143.317979,4.341977e+07,59.76,0.17,27.51,0.46,0.0,0.0,2.913830e+11
7,2009,"148,358.11","135,181",228.8362,132690.267032,7.236850e+07,60.83,0.17,26.59,0.49,0.0,0.0,2.817106e+11
8,2010,"164,828.81","149,301",241.0190,134984.309304,7.490822e+07,67.75,0.13,26.54,0.73,0.0,0.0,3.411048e+11
9,2011,"164,089.92","148,855",244.8782,94544.693780,5.146751e+07,61.93,0.19,28.27,0.86,0.0,0.0,3.708187e+11
